In [1]:
%load_ext autoreload
%autoreload 2
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import math
from time import time
import pickle
import pandas as pd
import numpy as np
from time import time

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score

In [3]:
import sys
sys.path.append('../src')
from preprocessing import *
from utils import *
from plotting import *

# Splitting the dataset

In [31]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']

df_db = group_datafiles_byID('../datasets/preprocessed/HT_Sensor_prep_metadata.dat', '../datasets/preprocessed/HT_Sensor_prep_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt,t0_delay,dt_delay
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,background,13.49,1.64,0.0,0.0
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,background,13.49,1.64,0.0,0.0
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,background,13.49,1.64,0.0,0.0
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,background,13.49,1.64,0.0,0.0
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,background,13.49,1.64,0.0,0.0


In [9]:
df_train, df_test = split_series_byID(0.75, df_db)
df_train, df_test = norm_train_test(df_train, df_test)
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values

../src\preprocessing.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[features_to_norm] = scaler.transform(df_train[features_to_norm])
C:\Users\David\anaconda3\lib\site-packages\pandas\core\indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
../src\preprocessing.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

# Basic Neural Network

In [6]:
def printResults(n_hid_layers,n_neur,accuracy,elapsed):
    print('========================================')
    print('Number of hidden layers:', n_hid_layers)
    print('Number of neurons per layer:', n_neur)
    print('Accuracy:', accuracy)
    print('Time (minutes):', (elapsed)/60)
    
def printScores(xtest,ytest,clf):
    xback, yback = xtest[ytest=='background'], ytest[ytest=='background']
    print('Score del background:', clf.score(xback,yback))
    xrest, yrest = xtest[ytest!='background'], ytest[ytest!='background']
    print('Score del resto:', clf.score(xrest,yrest))
    num_back = len(yback)
    num_wine = len(yrest[yrest=='wine'])
    num_banana = len(yrest[yrest=='banana'])
    func = lambda x: 1/num_back if x=='background' else (1/num_wine if x=='wine' else 1/num_banana)
    weights = np.array([func(x) for x in ytest])
    # Score donde las tres clases ponderan igual
    print('Score con pesos:', clf.score(xtest,ytest,weights))
    print('========================================')

In [9]:
# NN with 2 hidden layers and 15 neurons per layer
xtrain, ytrain, xtest, ytest = split_train_test(df_db,0.75)
start = time.time()

clf = MLPClassifier(hidden_layer_sizes=(15,15))
clf.fit(xtrain,ytrain)
score = clf.score(xtest,ytest)

final = time.time()

printResults(2,15,score,final-start)

C:\Users\David\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Number of hidden layers: 2
Number of neurons per layer: 15
Accuracy: 0.7868892049266106
Time (minutes): 10.664583432674409


In [11]:
# Adding early stopping and more iterations
xtrain, ytrain, xtest, ytest = split_train_test(df_db,0.75)
start = time.time()

clf = MLPClassifier(hidden_layer_sizes=(15,15),early_stopping=True,max_iter=2000)
clf.fit(xtrain,ytrain)
score = clf.score(xtest,ytest)

final = time.time()

printResults(2,15,score,final-start)

Number of hidden layers: 2
Number of neurons per layer: 15
Accuracy: 0.77063573442707
Time (minutes): 4.929224785168966


In [12]:
# Análisis del score
print('Proporcion de background:',len(ytest[ytest=='background'])/len(ytest))
printScores(xtest,ytest,clf)

Proporcion de background: 0.7930098051781594
Score del background: 0.9041396866445102
Score del resto: 0.25916250628894305
Score con pesos: 0.4761530892089738


Demasiado sesgo hacia el background, hay que reducirlo aunque el score baje

# Removing excess of background

In [18]:
# prop: ejemplos que no son background que habrá por cada ejemplo de background
def remove_bg(df,prop=2):
    new_df = df[df['class']!='background'].copy()
    useful_samples = new_df.shape[0]
    new_df = new_df.append(df[df['class']=='background'].sample(n=int(useful_samples/2)).copy())
    return new_df

In [18]:
# Para evitar el sesgo quitamos elementos clasificados como background, pero solo en el train set
df_train, df_test = split_series_byID(0.75, df_db)
df_train, df_test = norm_train_test(df_train, df_test)
df_train = remove_bg(df_train)

../src\preprocessing.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[features_to_norm] = scaler.transform(df_train[features_to_norm])
C:\Users\David\anaconda3\lib\site-packages\pandas\core\indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
../src\preprocessing.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [19]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values

In [20]:
start = time()

clf = MLPClassifier(hidden_layer_sizes=(15,15),early_stopping=True,max_iter=2000)
clf.fit(xtrain,ytrain)
score = clf.score(xtest,ytest)

final = time()

printResults(2,15,score,final-start)

Number of hidden layers: 2
Number of neurons per layer: 15
Accuracy: 0.6561803382403519
Time (minutes): 1.2229466478029887


In [21]:
# Análisis del score
printScores(xtest,ytest,clf)

Score del background: 0.7097241472743
Score del resto: 0.42049129567191906
Score con pesos: 0.502403749329069


Aunque se ponga la misma cantidad de background que de bananas o wine sigue habiendo un sesgo hacia el background.

# Hyperparameter analysis

In [35]:
xtrain, ytrain, xtest, ytest = split_train_test(df_db,0.75)

start_total = time.time()

for n_hid_layers in range(2,5):
    for n_neur in [10,20,40]:
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        start = time.time()

        clf_nn = MLPClassifier(
                    hidden_layer_sizes = tup,
                    max_iter=2000,
                    early_stopping=True
                )

        clf_nn.fit(xtrain, ytrain)
        ypred = clf_nn.predict(xtest)
        final = time.time()
        metric_report(ytest, ypred)
        print('\n====>Tiempo transcurrido (minutos):', (final-start)/(60))
        print('Number of hidden layers:', n_hid_layers)
        print('Number of neurons per layer:', n_neur)
        

end_total = time()
print('\n====> Total tiempo transcurrido (horas):', (end_total-start_total)/(60*60))

TEST SET PROPORTIONS:
	Real background percentage: 0.8247409627112922
	Real banana percentage: 0.0280836516344145
	Real wine percentage: 0.14717538565429336
------------------------------------------
Accuracy: 0.8468778356933336
Recall on background: 0.924579090118213
Recall on banana: 0.35106702735196876
Recall on wine: 0.5060652117805626
F1-score: 0.8474634761377202

====>Tiempo transcurrido (minutos): 1.5095600644747416
Number of hidden layers: 2
Number of neurons per layer: 10
TEST SET PROPORTIONS:
	Real background percentage: 0.8247409627112922
	Real banana percentage: 0.0280836516344145
	Real wine percentage: 0.14717538565429336
------------------------------------------
Accuracy: 0.852157764787811
Recall on background: 0.9345529911468196
Recall on banana: 0.23399458972046888
Recall on wine: 0.508388058845459
F1-score: 0.8486246905930726

====>Tiempo transcurrido (minutos): 4.483641095956167
Number of hidden layers: 2
Number of neurons per layer: 20
TEST SET PROPORTIONS:
	Real ba

TypeError: 'module' object is not callable

# Two Neural Networks

## 1. Classify background

In [24]:
def printScoresBack(xtest,ytest,clf):
    xback, yback = xtest[ytest=='background'], ytest[ytest=='background']
    print('Score del background:', clf.score(xback,yback))
    xrest, yrest = xtest[ytest!='background'], ytest[ytest!='background']
    print('Score del resto:', clf.score(xrest,yrest))
    num_back = len(yback)
    num_rest = len(ytest)-num_back
    func = lambda x: 1/num_back if x=='background' else 1/num_rest
    weights = np.array([func(x) for x in ytest])
    # Score donde las tres clases ponderan igual
    print('Score con pesos:', clf.score(xtest,ytest,weights))
    print('========================================')

In [7]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db.loc[df_db['class']!='background','class'] = 'not-background'
df_db[df_db['class']!='background'].head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
3429,13.490021,12.8102,10.3665,10.4529,11.6742,13.4941,13.2749,8.30531,9.04553,26.4234,59.4725,0,07-04-15,not-background,13.49,1.64
3430,13.490309,12.8097,10.3655,10.4523,11.6734,13.4934,13.2740,8.30527,9.04545,26.4241,59.4745,0,07-04-15,not-background,13.49,1.64
3431,13.490587,12.8088,10.3645,10.4516,11.6731,13.4930,13.2730,8.30523,9.04538,26.4246,59.4763,0,07-04-15,not-background,13.49,1.64
3432,13.490865,12.8080,10.3638,10.4508,11.6727,13.4922,13.2719,8.30520,9.04516,26.4251,59.4779,0,07-04-15,not-background,13.49,1.64
3433,13.491144,12.8078,10.3628,10.4503,11.6722,13.4914,13.2708,8.30517,9.04511,26.4256,59.4793,0,07-04-15,not-background,13.49,1.64


In [ ]:
# Primero probamos a no quitar el exceso de background

df_train, df_test = split_series_byID(0.75, df_db)
df_train, df_test = norm_train_test(df_train, df_test)
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values

start_total = time.time()

for n_hid_layers in range(2,5):
    for n_neur in [10,20,40]:
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        start = time.time()

        clf_nn = MLPClassifier(
                    hidden_layer_sizes = tup,
                    max_iter=2000,
                    early_stopping=True
                )

        clf_nn.fit(xtrain, ytrain)
        ypred = clf_nn.predict(xtest)
        final = time.time()
        metric_report(ytest, ypred)
        print('\n====>Tiempo transcurrido (minutos):', (end_total-start_total)/(60))
        
        

end_total = time.time()
print('\n====> Total tiempo transcurrido (horas):', (end_total-start_total)/(60*60))
# En más de la mitad de ocasiones aquellos datos que no son background son clasificados erroneamente.
# Veamos si es cuestión de quitar background.

In [32]:
# Ahora, lo mismo quitando el exceso de background
df_train, df_test = split_series_byID(0.75, df_db)
df_train = remove_bg(df_train,prop=1)
df_train, df_test = norm_train_test(df_train, df_test)
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values

start_total = time()

for n_hid_layers in range(2,5):
    for n_neur in [10,20,40]:
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        start = time()

        clf_nn = MLPClassifier(
                    hidden_layer_sizes = tup,
                    max_iter=2000,
                    early_stopping=True,
                    shuffle=True
                )

        clf_nn.fit(xtrain, ytrain)
        score = clf_nn.score(xtest, ytest)
        final = time()
        
        printResults(n_hid_layers,n_neur,score,final-start)
        printScoresBack(xtest,ytest,clf_nn)
        

end_total = time()
print('\n====> Total tiempo transcurrido (horas):', (end_total-start_total)/(60*60))

Number of hidden layers: 1
Number of neurons per layer: 5
Accuracy: 0.5801633486905403
Time (minutes): 0.4300204555193583
Score del background: 0.5223014632760336
Score del resto: 0.7499749841566326
Score con pesos: 0.6361382237163332
Number of hidden layers: 1
Number of neurons per layer: 10
Accuracy: 0.566574975523127
Time (minutes): 0.21988988320032757
Score del background: 0.5181815598806648
Score del resto: 0.7085987792268437
Score con pesos: 0.6133901695537544
Number of hidden layers: 1
Number of neurons per layer: 15
Accuracy: 0.5998508076308506
Time (minutes): 0.598010802268982
Score del background: 0.5622503196476772
Score del resto: 0.7101997932023615
Score con pesos: 0.6362250564250194
Number of hidden layers: 1
Number of neurons per layer: 20
Accuracy: 0.6004441863717856
Time (minutes): 0.21585826476415
Score del background: 0.5434919732916608
Score del resto: 0.7675861378873287
Score con pesos: 0.6555390555894948
Number of hidden layers: 1
Number of neurons per layer: 40
A

## 2. Classify wine and bananas

In [34]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db = df_db[df_db['class']!='background']
df_db.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
3429,13.490021,12.8102,10.3665,10.4529,11.6742,13.4941,13.2749,8.30531,9.04553,26.4234,59.4725,0,07-04-15,banana,13.49,1.64
3430,13.490309,12.8097,10.3655,10.4523,11.6734,13.4934,13.2740,8.30527,9.04545,26.4241,59.4745,0,07-04-15,banana,13.49,1.64
3431,13.490587,12.8088,10.3645,10.4516,11.6731,13.4930,13.2730,8.30523,9.04538,26.4246,59.4763,0,07-04-15,banana,13.49,1.64
3432,13.490865,12.8080,10.3638,10.4508,11.6727,13.4922,13.2719,8.30520,9.04516,26.4251,59.4779,0,07-04-15,banana,13.49,1.64
3433,13.491144,12.8078,10.3628,10.4503,11.6722,13.4914,13.2708,8.30517,9.04511,26.4256,59.4793,0,07-04-15,banana,13.49,1.64


In [36]:
xtrain, ytrain, xtest, ytest = split_train_test(df_db,0.75)

start_total = time()

for n_hid_layers in range(1,5):
    for n_neur in [5,10,15,20,40]:
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        start = time()

        clf_nn = MLPClassifier(
                    hidden_layer_sizes = tup,
                    max_iter=2000,
                    early_stopping=True,
                    shuffle=True
                )

        clf_nn.fit(xtrain, ytrain)
        score = clf_nn.score(xtest, ytest)
        final = time()
        
        printResults(n_hid_layers,n_neur,score,final-start)
        

end_total = time()
print('\n====> Total tiempo transcurrido (horas):', (end_total-start_total)/(60*60))

Number of hidden layers: 1
Number of neurons per layer: 5
Accuracy: 0.7908191826833364
Time (minutes): 0.12984011173248292
Number of hidden layers: 1
Number of neurons per layer: 10
Accuracy: 0.7634353424146296
Time (minutes): 0.47473315795262655
Number of hidden layers: 1
Number of neurons per layer: 15
Accuracy: 0.7635053181563725
Time (minutes): 0.5810888727506002
Number of hidden layers: 1
Number of neurons per layer: 20
Accuracy: 0.7582804627729054
Time (minutes): 0.370017139116923
Number of hidden layers: 1
Number of neurons per layer: 40
Accuracy: 0.7931517074080985
Time (minutes): 1.0173205216725667
Number of hidden layers: 2
Number of neurons per layer: 5
Accuracy: 0.6941593580891957
Time (minutes): 0.43400735457738243
Number of hidden layers: 2
Number of neurons per layer: 10
Accuracy: 0.6807473409218138
Time (minutes): 0.7764243205388387
Number of hidden layers: 2
Number of neurons per layer: 15
Accuracy: 0.7795297630154879
Time (minutes): 1.0063043196996053
Number of hidden

# 3. Merge the 2 NN

In [16]:
class doubleNN:
    def __init__(self, n_hid_layers, n_neur):
        self.hid_layers = n_hid_layers
        self.neur = n_neur
        
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)
        self.backNN = MLPClassifier(
                        hidden_layer_sizes = tup,
                        max_iter=2000,
                        early_stopping=True,
                        shuffle=True
                    )
        self.wineNN = MLPClassifier(
                        hidden_layer_sizes = tup,
                        max_iter=2000,
                        early_stopping=True,
                        shuffle=True
                    )
    
    def fit_bg(self, xtrain, ytrain):
        ytrain_copy = np.array([x if x=='background' else 'not-background' for x in ytrain])
        self.backNN.fit(xtrain, ytrain_copy)
    
    def fit_wine(self,xtrain,ytrain):
        self.wineNN.fit(xtrain, ytrain)
    
    def predict(self,xtest):
        ypred = self.backNN.predict(xtest)
        ypred[ypred=='not-background'] = self.wineNN.predict(xtest[ypred=='not-background'])
        return ypred
    
    def score(self,xtest,ytest):
        ypred = self.predict(ytest)
        score = np.sum(np.equal(ypred,ytest))/len(ytest)
        return score

In [17]:
# With all the background
xtrain, ytrain, xtest, ytest = split_train_test(df_db,0.75)

start_total = time.time()

for n_hid_layers in range(2,4):
    for n_neur in [10,20]:
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        start = time.time()

        clf_nn = doubleNN(2,20)
        clf_nn.fit_bg(xtrain, ytrain)

        xtrain_notbg = xtrain[ytrain != 'background']
        ytrain_notbg = ytrain[ytrain != 'background']
        clf_nn.fit_wine(xtrain_notbg, ytrain_notbg)
        ypred = clf_nn.predict(xtest)
        
        final = time.time()
        metric_report(ytest, ypred)
        print('\n====>Tiempo transcurrido (minutos):', (final-start)/(60))
        print('Number of hidden layers:', n_hid_layers)
        print('Number of neurons per layer:', n_neur)
        
        

end_total = time.time()
print('\n====> Total tiempo transcurrido (horas):', (end_total-start_total)/(60*60))

TEST SET PROPORTIONS:
	Real background percentage: 0.7869137667040765
	Real banana percentage: 0.03354791874402867
	Real wine percentage: 0.17953831455189476
------------------------------------------
Accuracy: 0.782324448134442
Recall on background: 0.8975362249685864
Recall on banana: 0.1738589727645815
Recall on wine: 0.39104873004438456
F1-score: 0.7820318012758023

====>Tiempo transcurrido (minutos): 3.209248713652293
Number of hidden layers: 2
Number of neurons per layer: 10
TEST SET PROPORTIONS:
	Real background percentage: 0.7869137667040765
	Real banana percentage: 0.03354791874402867
	Real wine percentage: 0.17953831455189476
------------------------------------------
Accuracy: 0.7321673682761268
Recall on background: 0.8322544045214275
Recall on banana: 0.14376321353065538
Recall on wine: 0.4034345734668743
F1-score: 0.741516705842115

====>Tiempo transcurrido (minutos): 4.176293102900187
Number of hidden layers: 2
Number of neurons per layer: 20
TEST SET PROPORTIONS:
	Real 

In [21]:
# Removing background
df_train, df_test = split_series_byID(0.75, df_db)
df_train = remove_bg(df_train,prop=1)
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values

start_total = time.time()

for n_hid_layers in range(2,4):
    for n_neur in [10,20]:
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        start = time.time()

        clf_nn = doubleNN(2,20)
        clf_nn.fit_bg(xtrain, ytrain)

        xtrain_notbg = xtrain[ytrain != 'background']
        ytrain_notbg = ytrain[ytrain != 'background']
        clf_nn.fit_wine(xtrain_notbg, ytrain_notbg)
        ypred = clf_nn.predict(xtest)
        
        final = time.time()
        metric_report(ytest, ypred)
        print('\n====>Tiempo transcurrido (minutos):', (final-start)/(60))
        print('Number of hidden layers:', n_hid_layers)
        print('Number of neurons per layer:', n_neur)
        
        

end_total = time.time()
print('\n====> Total tiempo transcurrido (horas):', (end_total-start_total)/(60*60))

TEST SET PROPORTIONS:
	Real background percentage: 0.8057390890907302
	Real banana percentage: 0.09305833360658425
	Real wine percentage: 0.1012025773026855
------------------------------------------
Accuracy: 0.7004500442666491
Recall on background: 0.7943239682370117
Recall on banana: 0.12482381959126145
Recall on wine: 0.4823619942489166
F1-score: 0.7126424791959668

====>Tiempo transcurrido (minutos): 1.6882099628448486
Number of hidden layers: 2
Number of neurons per layer: 10
TEST SET PROPORTIONS:
	Real background percentage: 0.8057390890907302
	Real banana percentage: 0.09305833360658425
	Real wine percentage: 0.1012025773026855
------------------------------------------
Accuracy: 0.7359658327048562
Recall on background: 0.8590555547077287
Recall on banana: 0.163715644820296
Recall on wine: 0.28216759143007575
F1-score: 0.7346641638156696

====>Tiempo transcurrido (minutos): 1.2407544692357382
Number of hidden layers: 2
Number of neurons per layer: 20
TEST SET PROPORTIONS:
	Real

# Creating Windows

In [4]:
# with open('../datasets/preprocessed/window120_dataset.pkl', 'wb') as f:
#     pickle.dump(win_df, f)
    
win_df = pd.read_pickle('../datasets/preprocessed/window120_dataset.pkl')

In [27]:
xtrain, ytrain, xtest, ytest = split_train_test(win_df,0.75)

start = time.time()

clf_nn = MLPClassifier(
            hidden_layer_sizes = (32,16),
            max_iter=2000,
            early_stopping=True,
            shuffle=True,
            alpha=0.01,
            learning_rate_init=0.01
        )

clf_nn.fit(xtrain, ytrain)
ypred = clf_nn.predict(xtest)
final = time.time()
metric_report(ytest, ypred)
print('\n====>Tiempo transcurrido (minutos):', (final-start)/(60))

TEST SET PROPORTIONS:
	Real background percentage: 0.7348271825570399
	Real banana percentage: 0.09707520485230221
	Real wine percentage: 0.1680976125906578
------------------------------------------
Accuracy: 0.7699655489875068
Recall on background: 0.8834440690265085
Recall on banana: 0.24589108698450843
Recall on wine: 0.5765513481363996
F1-score: 0.7671143839504504

====>Tiempo transcurrido (minutos): 4.107463216781616


In [41]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']

# Varía ciertos hiperparámetros con ventanas e imprime los resultados más relevantes
def hyper_sim(win_df,num_val,n_hid_layers,n_neur,alpha):
    errs_acc = []
    errs_f1 = []
    rec_ban = []
    loss = []
    for i in range(num_val):
        df_train, df_test = split_series_byID(0.75, win_df)
        df_train, df_test = norm_train_test(df_train,df_test,features_to_norm=features)
        xtrain, ytrain = df_train[features].values, df_train['class'].values
        xtest, ytest = df_test[features].values, df_test['class'].values
        
        tup = []
        for i in range(n_hid_layers):
            tup.append(n_neur)
        tup = tuple(tup)

        clf_nn = MLPClassifier(
                hidden_layer_sizes=tup,
                max_iter=2000,
                early_stopping=True,
                shuffle=True,
                alpha=alpha,
                learning_rate='adaptive'
            )

        clf_nn.fit(xtrain, ytrain)
        ypred = clf_nn.predict(xtest)
        errs_acc.append(accuracy_score(ytest,ypred))
        errs_f1.append(f1_score(ytest,ypred,average='weighted'))
        rec_ban.append(np.sum(np.logical_and(ytest=='banana',ypred=='banana'))/np.sum(ytest=='banana'))
        loss.append(clf_nn.loss_)

    errs_acc = np.array(errs_acc)
    errs_f1 = np.array(errs_f1)
    rec_ban = np.array(rec_ban)
    loss = np.array(loss)
    print('Train loss:',np.mean(loss),'+-',np.std(loss))
    print('Accuracy:',np.mean(errs_acc),'+-',np.std(errs_acc))
    print('F1-score:',np.mean(errs_f1),'+-',np.std(errs_f1))
    print('Recall bananas:',np.mean(rec_ban),'+-',np.std(rec_ban))

In [43]:
for alpha in [0.1,0.01,0.001]:
    print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>')
    print('Alpha:',alpha)
    for n_hid_layers in range(1,4):
        print('##############################################')
        print('\t Hidden layers:',n_hid_layers)
        for n_neur in [4,8,16]:
            print('==============================================')
            print('\t \t Neurons per layer:',n_neur)
            hyper_sim(win_df,3,n_hid_layers,n_neur,alpha)
            print('==============================================')

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>
Alpha: 0.1
##############################################
	 Hidden layers: 1
	 	 Neurons per layer: 4
Train loss: 0.30631837585880617 +- 0.023258576145879454
Accuracy: 0.8651469754197983 +- 0.028237533883041985
F1-score: 0.8449596309548207 +- 0.037541891251813155
Recall bananas: 0.24503317567532323 +- 0.07087260076560559
	 	 Neurons per layer: 8
Train loss: 0.24434164283115822 +- 0.010833188926485271
Accuracy: 0.845557828927527 +- 0.01047009214209502
F1-score: 0.8361074906063378 +- 0.007906986730144373
Recall bananas: 0.24727647351281004 +- 0.02763976878187987
	 	 Neurons per layer: 16
Train loss: 0.1686303778607473 +- 0.009096902594708849
Accuracy: 0.8361197586130197 +- 0.03367222583611265
F1-score: 0.815334460775949 +- 0.04026998400344681
Recall bananas: 0.23011923008334897 +- 0.14783895790099125
##############################################
	 Hidden layers: 2
	 	 Neurons per layer: 4
Train loss: 0.2683550733826729 +- 0.0150924051029735

Train loss: 0.12597540203156024 +- 0.00652878529757319
Accuracy: 0.7904499930320901 +- 0.023865778754145743
F1-score: 0.7827879639319401 +- 0.03188087906654341
Recall bananas: 0.2922362230825459 +- 0.0655589244980011
	 	 Neurons per layer: 16
Train loss: 0.03444057209840557 +- 0.009370458449644988
Accuracy: 0.7361013256213628 +- 0.06939709523090921
F1-score: 0.7612058243163494 +- 0.05328075601856744
Recall bananas: 0.17005808615439136 +- 0.06667497803794473
##############################################
	 Hidden layers: 3
	 	 Neurons per layer: 4
Train loss: 0.220272001012929 +- 0.019263988855455303
Accuracy: 0.838424164046585 +- 0.04145891541350635
F1-score: 0.8241867514825949 +- 0.03929488230010825
Recall bananas: 0.293826786185553 +- 0.06591626382535526
	 	 Neurons per layer: 8
Train loss: 0.11079169339472705 +- 0.01051375322682003
Accuracy: 0.7898933775384466 +- 0.029628924434023145
F1-score: 0.7919980310185984 +- 0.030750604723138307
Recall bananas: 0.2758330508080667 +- 0.1093578

In [5]:
# Nos quedamos con:
# alpha: 0.01
# hidden_layers: 3
# n_neurons: 4
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']


errs_acc = []
errs_f1 = []
rec_ban = []
for i in range(5):
    df_train, df_test = split_series_byID(0.75, win_df)
    df_train, df_test = norm_train_test(df_train,df_test,features_to_norm=features)
    xtrain, ytrain = df_train[features].values, df_train['class'].values
    xtest, ytest = df_test[features].values, df_test['class'].values

    clf_nn = MLPClassifier(
        hidden_layer_sizes=(4,4,4),
        max_iter=2000,
        early_stopping=True,
        shuffle=True,
        alpha=0.01,
        learning_rate='adaptive'
        )

    bag = BaggingClassifier(base_estimator=clf_nn,n_estimators=100,n_jobs=3)

    bag.fit(xtrain, ytrain)
    ypred = bag.predict(xtest)
    metric_report(ytest, ypred)
    errs_acc.append(accuracy_score(ytest,ypred))
    errs_f1.append(f1_score(ytest,ypred,average='weighted'))
    rec_ban.append(np.sum(np.logical_and(ytest=='banana',ypred=='banana'))/np.sum(ytest=='banana'))

errs_acc = np.array(errs_acc)
errs_f1 = np.array(errs_f1)
rec_ban = np.array(rec_ban)
print('Accuracy:',np.mean(errs_acc),'+-',np.std(errs_acc))
print('F1-score:',np.mean(errs_f1),'+-',np.std(errs_f1))
print('Recall bananas:',np.mean(rec_ban),'+-',np.std(rec_ban))

TEST SET PROPORTIONS:
	Real background percentage: 0.753916922093646
	Real banana percentage: 0.1666419116290165
	Real wine percentage: 0.07944116627733744
------------------------------------------
Accuracy: 0.7947987415439042
Recall on background: 0.9468905034536695
Recall on banana: 0.17899200518582542
Recall on wine: 0.6431728045325779
F1-score: 0.7606228511303869
TEST SET PROPORTIONS:
	Real background percentage: 0.7767477738985424
	Real banana percentage: 0.11219462168537171
	Real wine percentage: 0.11105760441608589
------------------------------------------
Accuracy: 0.8900125294844184
Recall on background: 0.9690876109344324
Recall on banana: 0.33550592162785153
Recall on wine: 0.8971373509455173
F1-score: 0.8764919643302215
TEST SET PROPORTIONS:
	Real background percentage: 0.8233239114555213
	Real banana percentage: 0.087355779482689
	Real wine percentage: 0.08932030906178967
------------------------------------------
Accuracy: 0.8873276499598474
Recall on background: 0.9677

In [10]:
with open('../datasets/preprocessed/nn_optimal.pkl', 'wb') as f:
    pickle.dump(bag, f)